Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

print('Before reshaping:')
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)


def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print('\nAfter reshaping')
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Before reshaping:
Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)

After reshaping
Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.



In [5]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  beta = 0.01
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss + (beta*tf.nn.l2_loss(weights)))
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 16.735006
Training accuracy: 12.7%
Validation accuracy: 14.7%
Loss at step 100: 1.399714
Training accuracy: 73.8%
Validation accuracy: 72.6%
Loss at step 200: 0.816974
Training accuracy: 79.4%
Validation accuracy: 76.9%
Loss at step 300: 0.658945
Training accuracy: 82.3%
Validation accuracy: 79.8%
Loss at step 400: 0.617082
Training accuracy: 83.7%
Validation accuracy: 81.1%
Loss at step 500: 0.605314
Training accuracy: 84.3%
Validation accuracy: 81.5%
Loss at step 600: 0.601732
Training accuracy: 84.6%
Validation accuracy: 81.6%
Loss at step 700: 0.600533
Training accuracy: 84.6%
Validation accuracy: 81.7%
Loss at step 800: 0.600076
Training accuracy: 84.6%
Validation accuracy: 81.8%
Test accuracy: 88.8%


## Let's try regularization on Neural Network

In [7]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  # Hidden layer
  hidden_layer1_size = 1024
  
  # Variables
  # Connecting Input with Hidden Layer
  l1_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer1_size]))
  l1_biases = tf.Variable(tf.zeros([hidden_layer1_size]))
  # Connecting Hidden Layer with Output layer
  o1_weights = tf.Variable(
    tf.truncated_normal([hidden_layer1_size, num_labels]))
  o1_biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # First Linear
  logits_1 = tf.matmul(tf_train_dataset, l1_weights) + l1_biases
  # ReLU
  relus = tf.nn.relu(logits_1)
  # Second Linear
  logits_2 = tf.matmul(relus, o1_weights) + o1_biases
  # Output
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_2))
  
  # Optimizer.
  beta = 0.01
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss + (beta*tf.nn.l2_loss(o1_weights)))
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_2)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, l1_weights) + l1_biases), o1_weights) + o1_biases)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, l1_weights) + l1_biases), o1_weights) + o1_biases) 

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 444.635406
Minibatch accuracy: 9.4%
Validation accuracy: 43.8%
Minibatch loss at step 500: 26.208252
Minibatch accuracy: 69.5%
Validation accuracy: 71.8%
Minibatch loss at step 1000: 9.500387
Minibatch accuracy: 75.8%
Validation accuracy: 71.8%
Minibatch loss at step 1500: 3.465525
Minibatch accuracy: 76.6%
Validation accuracy: 78.4%
Minibatch loss at step 2000: 2.310279
Minibatch accuracy: 78.1%
Validation accuracy: 79.1%
Minibatch loss at step 2500: 4.681852
Minibatch accuracy: 71.1%
Validation accuracy: 78.8%
Minibatch loss at step 3000: 3.817307
Minibatch accuracy: 77.3%
Validation accuracy: 78.7%
Test accuracy: 85.4%


---

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
Problem 3 and Problem 4
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?



In [10]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  # Hidden layer
  hidden_layer1_size = 1024
  hidden_layer2_size = 500
  
  # Variables
  # Connecting Input with Hidden Layer
  l1_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer1_size], stddev = 0.0517))
  l1_biases = tf.Variable(tf.zeros([hidden_layer1_size]))
  
  l2_weights = tf.Variable(
    tf.truncated_normal([hidden_layer1_size, hidden_layer2_size], stddev = 0.0417))
  l2_biases = tf.Variable(tf.zeros([hidden_layer2_size])) 
    
  # Connecting Hidden Layer with Output layer
  o1_weights = tf.Variable(
    tf.truncated_normal([hidden_layer2_size, num_labels], stddev = 0.317))
  o1_biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # First Linear
  logits_1 = tf.matmul(tf_train_dataset, l1_weights) + l1_biases
  # ReLU
  relus_1 = tf.nn.relu(logits_1)
  
  logits_2 = tf.matmul(relus_1, l2_weights) + l2_biases
  relus_2 = tf.nn.relu(logits_2)
    
  logits_3 = tf.matmul(relus_2, o1_weights) + o1_biases
  # Output
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_3))


  logits_1v = tf.matmul(tf_valid_dataset, l1_weights) + l1_biases
  # ReLU
  relus_1v = tf.nn.relu(logits_1v)
  logits_2v = tf.matmul(relus_1v, l2_weights) + l2_biases
  relus_2v = tf.nn.relu(logits_2v) 
  logits_3v = tf.matmul(relus_2v, o1_weights) + o1_biases

  logits_1t = tf.matmul(tf_test_dataset, l1_weights) + l1_biases
  relus_1t = tf.nn.relu(logits_1t)
  logits_2t = tf.matmul(relus_1t, l2_weights) + l2_biases
  relus_2t = tf.nn.relu(logits_2t) 
  logits_3t = tf.matmul(relus_2t, o1_weights) + o1_biases
  
  # Optimizer.
  beta = 0.01
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss + (beta*tf.nn.l2_loss(o1_weights)))
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_3)
  valid_prediction = tf.nn.softmax(logits_3v)
  test_prediction = tf.nn.softmax(logits_3t)

In [11]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.709104
Minibatch accuracy: 4.7%
Validation accuracy: 31.2%
Minibatch loss at step 500: 0.563433
Minibatch accuracy: 78.9%
Validation accuracy: 84.8%
Minibatch loss at step 1000: 0.416706
Minibatch accuracy: 87.5%
Validation accuracy: 85.9%
Minibatch loss at step 1500: 0.455343
Minibatch accuracy: 87.5%
Validation accuracy: 86.9%
Minibatch loss at step 2000: 0.260309
Minibatch accuracy: 92.2%
Validation accuracy: 87.2%
Minibatch loss at step 2500: 0.427841
Minibatch accuracy: 88.3%
Validation accuracy: 88.1%
Minibatch loss at step 3000: 0.348575
Minibatch accuracy: 89.8%
Validation accuracy: 88.1%
Minibatch loss at step 3500: 0.379327
Minibatch accuracy: 88.3%
Validation accuracy: 87.7%
Minibatch loss at step 4000: 0.455971
Minibatch accuracy: 85.2%
Validation accuracy: 88.4%
Minibatch loss at step 4500: 0.262549
Minibatch accuracy: 91.4%
Validation accuracy: 88.6%
Minibatch loss at step 5000: 0.353079
Minibatch accuracy: 89.8%
Validation accuracy

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
